In [2]:
#!pip install -U twitter-client-py
import streamlit as st

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import requests
import pprint as ppp
from twitter_client_py import TwitterClient, AsyncTwitterClient

In [4]:
# Q de consultas que vamos (limite mensual = 1.000)
api_key = os.getenv('2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a')
client = AsyncTwitterClient(api_key=api_key, timeout=20, verbose=True)
rate_limit = client.rate_limit
print(f"Limit: {rate_limit.limit}, Remaining requests: {rate_limit.remaining}, Reset time: {rate_limit.reset}")

Limit: 0, Remaining requests: 0, Reset time: 0


La proxima celda corresponde al codigo para trabajar en una API con cantidad de consultas limitadas. Se me ocurre que lo que habria que hacer, para evitar bloqueos (o exceder las cuotas de consulta), seria tener varios user con varias apis, e ir tomando de a uno por vez.

In [5]:
url_tweets_search_api_01 = "https://twitter-x.p.rapidapi.com/search/"

user_search_phrase = 'Madrid'

querystring = {"query":user_search_phrase,"section":'latest',"limit":"20", 'lang':'en'} 			# "section" predeterminado sin posibilidad de cambiar por el cliente (para traer el sentimiento en tiempo real)
# el parametro "lang" no funciona, hay que resolverlo en el EDA (PROBAR CON LA API XCOM)

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter-x.p.rapidapi.com"
}

response_api_01 = requests.get(url_tweets_search_api_01, headers=headers, params=querystring)
entries_api_01 = response_api_01.json()['data']['search_by_raw_query']['search_timeline']['timeline']['instructions'][0]['entries']

def clean_entries_with_dates(list_of_elem):
    clean_data = []
    for element in list_of_elem:
        content = element.get('content',[])                                                                             # para extraer el contenido de 'content'
        item_content = content.get('itemContent',{})                                                                    # para extraer el contenido de 'itemContent'
        if 'tweet_results' not in item_content or 'result' not in item_content['tweet_results'] : continue              # que siga de largo con este elemento si no tiene 'tweet_result' o 'result'
        result = item_content['tweet_results']['result']                                                                # para extraer el contenido de 'tweet_results'
        if 'legacy' not in result or 'full_text' not in result['legacy']: continue                                      # que siga de largo con este elemento si no tiene 'full_text' o 'legacy'
        full_text = result['legacy']['full_text']                                                                       # para extraer el contenido de 'full_text'
        post_date = result['legacy']['created_at']
        clean_data.append((post_date, full_text))
    return clean_data

aux_prueba = clean_entries_with_dates(entries_api_01)
aux_prueba = pd.DataFrame(aux_prueba)
aux_prueba																						# Resultado final

,0,1
0,Wed Oct 09 14:28:47 +0000 2024,Vendo 2 entradas VIP para ver a Vanessa Valero...
1,Wed Oct 09 14:28:46 +0000 2024,@todoesmentiratv La PREGUNTA es ¿quién debe re...
2,Thu Sep 19 01:00:00 +0000 2024,Celebrate Prof. Oscar Garcia-Prada's 60th birt...
3,Wed Oct 09 14:28:46 +0000 2024,@maguire_utd1 He fit to play Madrid
4,Wed Oct 09 14:28:45 +0000 2024,@theMadridZone @CH14_ Man United gave Real Mad...
5,Wed Oct 09 14:28:44 +0000 2024,i jus realized i always wanted big baller bran...
6,Wed Oct 09 14:28:44 +0000 2024,And in #London no less ... As did Cristiano di...
7,Wed Oct 02 13:06:34 +0000 2024,⚔️ Only 0.01% can complete 😲 Claim your place ...
8,Wed Oct 09 14:28:42 +0000 2024,الوحدة فولهام\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون▬\n◗EWW◖
9,Wed Oct 09 14:28:40 +0000 2024,الوحدة بورتو ماينز\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون▬\...


In [6]:
aux_prueba.to_csv('aux_prueba.csv')

In [7]:
response_api_01.json()

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
       'entries': [{'entryId': 'tweet-1844022199155581430',
         'sortIndex': '1844022206141562880',
         'content': {'entryType': 'TimelineTimelineItem',
          '__typename': 'TimelineTimelineItem',
          'itemContent': {'itemType': 'TimelineTweet',
           '__typename': 'TimelineTweet',
           'tweet_results': {'result': {'__typename': 'Tweet',
             'rest_id': '1844022199155581430',
             'core': {'user_results': {'result': {'__typename': 'User',
                'id': 'VXNlcjoxMjExMDI2ODA2ODE3NDgwNzA0',
                'rest_id': '1211026806817480704',
                'affiliates_highlighted_label': {},
                'has_graduated_access': True,
                'is_blue_verified': False,
                'profile_image_shape': 'Circle',
                'legacy': {'can_dm': False,
                 'can_media_tag': False,
            